In [3]:
import numpy as np
import statistics
import pandas as pd
import pydicom

%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json

Using TensorFlow backend.


In [13]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    if ds.Modality != 'DX':
        print('Wrong Modality')
        return None

    if ds.BodyPartExamined != 'CHEST':
        print('Wrong Body Part')
        return None 
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size): 
    # todo
    
    img = (img-np.mean(img))/np.std(img)
    proc_img = np.resize(img,(1,224,224,3))    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
       
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred_prob = model.predict(img, batch_size = 32, verbose = True)
    if pred_prob > thresh:
        prediction = 'Pneumonia'
    else: 
        prediction = 'No Pneumonia'
        
    return prediction 

In [15]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'#path to saved model
weight_path = 'xray_class_my_model.best.hdf5'#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.43 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
1/1 [==============================] - 1s 650ms/step
Pneumonia
Load file test2.dcm ...
1/1 [==============================] - 1s 546ms/step
Pneumonia
Load file test3.dcm ...
1/1 [==============================] - 1s 551ms/step
Pneumonia
Load file test4.dcm ...
Wrong Body Part
Load file test5.dcm ...
Wrong Modality
Load file test6.dcm ...
1/1 [==============================] - 1s 556ms/step
Pneumonia
